## Parte 3: Construyendo un Sistema de Recomendacion con Feedback Implicito

En este ejercicio, desarrollaremos un sistema de recomendacion con feedback implicito utilizando la libreria     [implicit](https://github.com/benfred/implicit).

**Pero, a que nos referimos con feedback implicito?**

En el primer ejercicio abordamos el filtro colaborativo el cual se basa en la suposicion de que `usuarios similares gustan de las mismas cosas/items`. La matriz usuario-item, o "matriz de utilidad" es la piedra angular del filtrado colaborativo. En la matriz de utilidad las filas representan a los usuarios y las columnas representan a los items.



Las celdas de la matriz se llenan a partir del grado de preferencia de un usuario a un item determinado y esto se representa en cualquiera de las dos formas:
1. **Feedback explicito:** feedback directo hacia un item (por ejemplo el rating de una pelicula como lo vimos en el [Ejercicio 1](https://experiencia21.tec.mx/courses/481176/assignments/15386625?module_item_id=28379086))

2. **Feedback implicito:** comportamiento indirecto hacia un item (por ejemplo el historial de compra, el historial de navegacion o historial de busquedas)

El feedback implicito hace suposiciones sobre las preferencias del usuario a partir de las acciones hacia dichos items. Si retomamos el ejemplo si miraste todos los episodios de un show y viste todas las temporadas en una semana, entonces existe la elevada posibilidad de que te guste ese show. Sin embargo, si empiezas a mirar una serie y te detienes a la mitad del primer episodio, entonces es probable que se pueda asumir que no te haya gustado ese show.



### Paso 1: Agregando las Librerias

Estos seran las librerias que utilizaremos:

- [numpy](https://numpy.org/)
- [pandas](https://pandas.pydata.org/)
- [implicit](https://github.com/benfred/implicit)
- scipy (en especifico la clase **csr_matrix**)

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

import implicit

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Paso 2: Cargando los datos

Dado que ya te has familiarizado con el dataset de BeersRating de los ejercicios 1 y 2 en este ejercicio continuaremos utilizando este dataset que puede encontrar[aqui](https://grouplens.org/datasets/movielens/), o lo puedes descargar directamente de [aqui](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip). (Recuerda que estamos trabajando con los datasets `ml-latest-small.zip` )


In [2]:
df = pd.read_csv('equipo27/ratebeer.csv')

/var/folders/rj/5tnxc9fs7jv8kk0ngkbs5_z00000gp/T/ipykernel_30763/4253166023.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('equipo27/ratebeer.csv')


In [3]:
# Definimos una función para convertir "x/y" en un número flotante
def convert_to_float(fraction):
    num, denom = fraction.split('/')
    return float(num) / float(denom)


In [4]:
# Aplicamos la función a las columnas relevantes y añadimos nuevas columnas al DataFrame
df['review/appearance_value'] = df['review/appearance'].apply(convert_to_float)
df['review/aroma_value'] = df['review/aroma'].apply(convert_to_float)
df['review/palate_value'] = df['review/palate'].apply(convert_to_float)
df['review/taste_value'] = df['review/taste'].apply(convert_to_float)
df['review/overall_value'] = df['review/overall'].apply(convert_to_float)
df['review/overall_value_int'] = (df['review/overall_value'] * 10).astype(int)

In [5]:
df.head()

,beer/name,beer/beerId,beer/brewerId,beer/ABV,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/overall,review/time,review/profileName,review/text,review/appearance_value,review/aroma_value,review/palate_value,review/taste_value,review/overall_value,review/overall_value_int
0,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale (IPA),4/5,6/10,3/5,6/10,13/20,1157587200,hopdog,"On tap at the Springfield, PA location. Poured...",0.8,0.6,0.6,0.6,0.65,6
1,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale (IPA),4/5,6/10,4/5,7/10,13/20,1157241600,TomDecapolis,On tap at the John Harvards in Springfield PA....,0.8,0.6,0.8,0.7,0.65,6
2,John Harvards Cristal Pilsner,71716,8481,5,Bohemian Pilsener,4/5,5/10,3/5,6/10,14/20,958694400,PhillyBeer2112,"UPDATED: FEB 19, 2003 Springfield, PA. Ive nev...",0.8,0.5,0.6,0.6,0.70,7
3,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Klsch,2/5,4/10,2/5,4/10,8/20,1157587200,TomDecapolis,On tap the Springfield PA location billed as t...,0.4,0.4,0.4,0.4,0.40,4
4,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Klsch,2/5,4/10,2/5,4/10,8/20,1157587200,hopdog,"On tap at the Springfield, PA location. Poured...",0.4,0.4,0.4,0.4,0.40,4


In [6]:
base_rate = df[['review/profileName', 'beer/beerId', 'review/appearance_value', 'review/aroma_value', 'review/palate_value', 'review/taste_value', 'review/overall_value', 'review/overall_value_int', 'review/time']]
base_rate = base_rate.drop_duplicates(keep="last", subset=['review/profileName', 'beer/beerId'])
base_rate.shape

(2864206, 9)

In [7]:
base_beers = df[['beer/beerId', 'beer/name', 'beer/style', 'beer/brewerId', 'beer/ABV']]
base_beers = base_beers.drop_duplicates(keep="last", subset=['beer/beerId'])
base_beers.shape

(110647, 5)

In [8]:
ratings = base_rate
beers = base_beers

Revisamos el contenido de ratings

In [9]:
ratings.head()

,review/profileName,beer/beerId,review/appearance_value,review/aroma_value,review/palate_value,review/taste_value,review/overall_value,review/overall_value_int,review/time
0,hopdog,63836,0.8,0.6,0.6,0.6,0.65,6,1157587200
1,TomDecapolis,63836,0.8,0.6,0.8,0.7,0.65,6,1157241600
2,PhillyBeer2112,71716,0.8,0.5,0.6,0.6,0.70,7,958694400
3,TomDecapolis,64125,0.4,0.4,0.4,0.4,0.40,4,1157587200
4,hopdog,64125,0.4,0.4,0.4,0.4,0.40,4,1157587200


En este ejercicio, definiremos el rating de las cervezas como el numero de veces que un usuario las ha tomado. Por ejemplo, si hopdog (un usuario en nuestro dataset) le dio a la cerveza `John Harvards Simcoe IPA` un rating de 0.65 y a `John Harvards Fancy Lawnmower Beer` un rating de 0.4, podemos asumir que ha probado la cerveza  `John Harvards Simcoe IPA` unas 6 veces y la `John Harvards Fancy Lawnmower Beer` un total de 4 veces.

### NOTA ###

Es necesario realizar una limpieza del dataset antes de proceder con el ejercicio pues contiene registros duplicados que arrojaran un problema en el numero de registros de los titulos de las peliculas. 

Para lo cual es necesario eliminar duplicados que contiene el dataset de peliculas y proceder con el ejercicio

### Paso 3: Transformando los datos

Tal y como lo hicimos en el [Ejercicio 1](https://experiencia21.tec.mx/courses/481176/assignments/15386625?module_item_id=28379086), necesitamos transformar el dataframe de `ratings` a una matriz usuario-item donde las filas representan a los usuarios y las columnas representan a las cervezas. Las celdas en esta matriz contendran el feedback implicito que en este caso es el numero de veces que un usuario ha visto una pelicula.

La funcion  `create_X()` crea una matriz de dispersion **X** con 4 diccionarios de mapeo:

- **user_mapper:** mapea user id al user index
- **beer_mapper:** mapea beer id al beer index
- **user_inv_mapper:** mapea user index al user id
- **beer_inv_mapper:** mapea beer index al beer id

Necesitamos estos diccionario por que hay que mapear las filas y columnas con la matriz de utilidad que les corresponde al user ID con su beer ID respectivamente.

Esta matriz dispersa **usuario-item** es una matriz que se obtiene al `usar scipy.sparse.csr_matrix`que almacena los datos de una manera dispersa.

In [10]:
def create_X(dat_frame: pd.DataFrame):
    """
    Generates a sparse matrix from ratings dataframe.
    
    Args:
        df: pandas dataframe
    
    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        beer_mapper: dict that maps movie id's to movie indices
        beer_inv_mapper: dict that maps movie indices to movie id's
    """
    N = dat_frame['review/profileName'].astype(str).nunique()
    B = dat_frame['beer/beerId'].astype(str).nunique()

    user_mapper = dict(zip(np.unique(dat_frame["review/profileName"].astype(str)), list(range(N))))
    beer_mapper = dict(zip(np.unique(dat_frame["beer/beerId"].astype(str)), list(range(B))))
    
    user_inv_mapper = dict(zip(list(range(N)), np.unique(dat_frame["review/profileName"].astype(str))))
    beer_inv_mapper = dict(zip(list(range(B)), np.unique(dat_frame["beer/beerId"].astype(str))))
    
    user_index = [user_mapper[i] for i in dat_frame['review/profileName'].astype(str)]
    beer_index = [beer_mapper[i] for i in dat_frame['beer/beerId'].astype(str)]

    X = csr_matrix((dat_frame["review/overall_value_int"], (beer_index, user_index)), shape=(B, N))
    
    return X, user_mapper, beer_mapper, user_inv_mapper, beer_inv_mapper

In [11]:
valor = '<'
if valor in ratings['beer/beerId'].values:
    print(f"El valor {valor} está presente en la columna 'beer/beerId'.")

In [12]:
X, user_mapper, beer_mapper, user_inv_mapper, beer_inv_mapper = create_X(ratings)

### Creando los Mapeos de los nombres de las cervezas

Necesitamos traducir el nombre de una cerveza a partir de su indice en la matriz usuario-item y vice versa. Vamos a crear dos funciones que nos ayuden con esta traduccion.

- `get_beer_index()` - convierte el nombre de una cerveza a su indice. Hace uso de la funcion de comparacion de strings que se le pasan a [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) 
 para obtener el nombre de una cerveza que se le pase. Esto significa que no necesitamos saber la forma de escribir o el formato de una cerveza para obtener su indice.

- `get_beer_title()` - convierte el indice de una cerveza a su nombre.

In [13]:
from fuzzywuzzy import process

def beer_finder(title):
    all_titles = beers['beer/name'].tolist()
    closest_match = process.extractOne(title, all_titles)
    return closest_match[0]

beer_title_mapper = dict(zip(beers['beer/name'], beers['beer/beerId']))
beer_title_inv_mapper = dict(zip(beers['beer/beerId'], beers['beer/name']))

def get_beer_index(title):
    fuzzy_title = beer_finder(title)
    beer_id = beer_title_mapper[fuzzy_title]
    beer_idx = beer_mapper[str(beer_id)]
    return beer_idx

def get_beer_title(beer_idx): 
    beer_id = beer_inv_mapper[beer_idx]
    title = beer_title_inv_mapper[int(beer_id)]
    return title 

Vamos a probar esta funcion para obtener el indice de `John Harvards Cristal Pilsner`. 

In [14]:
get_beer_index('John Harvards Cristal Pilsner')

35047

Utilizemos el indice obtenido con la funcion `get_beer_title()`. Tendremos que obtener el nombre de John Harvards Cristal Pilsner.

In [15]:
get_beer_title(35047)

'John Harvards Cristal Pilsner'

Con esto podemos comprobar que las funciones nos permitiran interpretar las recomendaciones obtenidas del sistema.

### Paso 4: Construyendo el modelo de modelo de Recomendacion de Feedback Implicito

Una vez que hemos transformado nuestros datos ahora si podemos empezar a construir nuestro modelo de recomendacion.


La libreria [implicit](https://github.com/benfred/implicit) esta basada en un factorizacion de matrices (tomado del algebra lineal). Esto nos permite hallar caracteristicas
latentes que se esconden en las interacciones entre los usuarios y las cervazas. Estas caracteristicas latentes nos brindan una representacion mas compacta de los gustos
de los usuarios y la descripcion de un item. La factorizacion matricial es particularmente util para datos muy dispersos y puede mejorar la calidad de las recomendaciones
obtenidas. El algoritmo opera al factorizar la matris usuario-item en dos matrices:

- matriz usuario-factorers  (n_users, k)
- matriz item-factorers     (k, n_items)

Reduciremos las dimensiones de nuestra matriz original a nuestras dimensiones particulares. No es posible interpretar cada caracteristica latente $k$. Sin embargo,
podemos suponer que una caracteristica latente puede representar a los usuarios que gusten de un estilo India Pale Ale (IPA) de cerveza, mientras que otra caracteristica lantente
puede representar a cervezas independientes extranjeras.


$$X_{mn} \approx P_{mk} \times Q_{nk}^T = \hat{X}$$



En el caso de una factorizacion matricial tradicional como [SVD](https://www.freecodecamp.org/news/singular-value-decomposition-vs-matrix-factorization-in-recommender-systems-b1e99bc73599/) lo que hariamos seria intentar resolver la factorizacion de una sola vez, sin embargo esto resultaria muy costoso computacionalmente. Otra forma de atacar este problem es utilizando una tecnica denominada
[Minimos Cuadrados Alternos, Alternating Least Squares (ALS)](https://sophwats.github.io/2018-04-05-gentle-als.html). Ocupando ALS, podemos resolver una matriz de factores a la vez:

- Paso 1: Fijamos la matriz de factores de usuario (user-factor) y resolvemos la matriz de factores de elementos (item-factor)
- Paso 2: Fijamos la matriz de factores de elementos (item-factor) y resolvemos la matriz de factores de usuario (user-factor)

Al alternar los pasos 1 y 2 hasta que el producto punto de la matriz de factores de elementos (item-factor) y la matriz de factores de usuarios (user-item) es aproximadamente igual a la matrix original X (user-item). Este procedimiento es comptacionalmente menos costoso y puede ser parelelizado.

La libreria `implicit` implementa una factorizacion matricial utilizando ALS (puedes consultar los detalles [aqui](https://implicit.readthedocs.io/en/latest/als.html))

In [16]:
model = implicit.als.AlternatingLeastSquares(factors=50)


/opt/homebrew/anaconda3/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


Este modelo viene con algunos hyperparametros que deben ser ajustados para generar resultados optimos:

- los factores ($k$): numero de factores latentes,
- regularizacion ($\lambda$): evita que el modelo caiga en overfitting durante el entrenamiento

Para este ejercicio definiremos $k = 50$ y $\lambda = 0.01$ como los valores a utilizar. 

El siguiente paso ahora es ajustar nuestro modelo a la matriz user-item.


In [17]:
model.fit(X.T.tocsr())

100%|██████████| 15/15 [02:52<00:00, 11.51s/it]



Ahora pongamos a prueba las recomendaciones de nuestro modelo. Podemos utilizar el metodo `similar_items()` que nos muestra las cervezas mas relevantes dada una cerveza en especifico. De igual forma, podemos utilizar la funcion `get_beer_index()` para obtener el indice de la cerveza si es que es una cerveza que nos interesa a partir de las recomendaciones obtenidas.

In [18]:
beer_of_interest = 'Twains Saramonial Saison'

beer_index = get_beer_index(beer_of_interest)
related = model.similar_items(beer_index)
related

(array([  5146, 105258,   3961,   5270,  31584,  15137,  19122,  39767,
         37904, 102911], dtype=int32),
 array([1.0000001 , 0.9557607 , 0.9528838 , 0.9325972 , 0.92886746,
        0.9241434 , 0.920319  , 0.9086316 , 0.9071206 , 0.90202886],
       dtype=float32))

Lo que obtenemos de `similar_items()` no es facil de leer por lo que necesitamos de la funcion `get_beer_title()` para interpretar los resultados.

In [20]:
print(f"Por que porbaste la cerveza {beer_finder(beer_of_interest)} te pueden interesar las siguientes cervezas:")

for t, r in zip(related[0], related[1]):
    try:
        recommended_title = get_beer_title(t)
        if recommended_title != beer_finder(beer_of_interest):
            print(recommended_title)
    except:
        continue



Por que porbaste la cerveza Twains Saramonial Saison te pueden interesar las siguientes cervezas:
Twains Honest Lender Imperial Brown
Red Brick Solstice Roggenbock
5 Seasons Westside Big Matts ESB
5 Seasons North Pepe le Pew Skunky Saison
5 Seasons Westside Spring Bock
5 Seasons Damage Control  Pale Ale
Red Hare Gangway IPA
Heinzelmannchen Dunkel Weiss


Al usar el rating de los usuarios como feedback implicito, los resultados se ven bien. Intenta cambiando la variable `beer_of_interest`.

### Paso 5: Generando las recomendaciones del usuario

Una caracteristica interesante de `implicit` es que puedes obtener recomendaciones personalizadas para un usuario determinado. Intentemos ver los resultados con un usuario especifico de nuestro dataset.

In [21]:
user_id = 'PhillyBeer2112'

In [22]:
user_ratings = ratings[ratings['review/profileName']==user_id].merge(beers[['beer/beerId', 'beer/name']])
user_ratings = user_ratings.sort_values('review/overall_value_int', ascending=False)
print(f"El numero de cervezas rankeadas por el usuario {user_id} es de: {user_ratings['beer/beerId'].nunique()}")

El numero de cervezas rankeadas por el usuario PhillyBeer2112 es de: 2090


En este caso vemos que el usuario PhillyBeer2112 probó 2090 cervezas y el rating de su favoritas son:

In [23]:
user_ratings = ratings[ratings['review/profileName']==user_id].merge(beers[['beer/beerId', 'beer/name']])
user_ratings = user_ratings.sort_values('review/overall_value_int', ascending=False)
top_5 = user_ratings.head()
top_5

,review/profileName,beer/beerId,review/appearance_value,review/aroma_value,review/palate_value,review/taste_value,review/overall_value,review/overall_value_int,review/time,beer/name
755,PhillyBeer2112,98973,0.8,0.8,1.0,1.0,1.0,10,1275436800,Founders CBS (Canadian Breakfast Stout)
562,PhillyBeer2112,10908,1.0,1.0,1.0,1.0,1.0,10,1053475200,Oud Beersel Oude Kriek
53,PhillyBeer2112,5141,0.8,0.9,0.6,0.7,1.0,10,970358400,Custom Brewcrafters Pizza Plant Inferno Pod Ale
1655,PhillyBeer2112,13145,0.6,0.9,0.8,0.9,1.0,10,1275436800,Russian River Temptation
1399,PhillyBeer2112,5923,0.8,0.9,1.0,0.9,1.0,10,991699200,Dogfish Head World Wide Stout 2001/2003-Presen...


Las cervezas con el menor rating son:

In [24]:
bottom_5 = user_ratings[user_ratings['review/overall_value_int']<5].tail()
bottom_5

,review/profileName,beer/beerId,review/appearance_value,review/aroma_value,review/palate_value,review/taste_value,review/overall_value,review/overall_value_int,review/time,beer/name
1066,PhillyBeer2112,407,0.2,0.2,0.2,0.1,0.05,0,959644800,Miller High Life
1253,PhillyBeer2112,52080,0.4,0.5,0.2,0.1,0.05,0,1140220800,Michelob Celebrate Vanilla Oak
1059,PhillyBeer2112,7342,0.4,0.1,0.2,0.1,0.05,0,1055808000,Olde English HG800 / Olde English 800 7.9%
1256,PhillyBeer2112,107964,0.2,0.1,0.2,0.1,0.05,0,1281225600,Budweiser Select 55
596,PhillyBeer2112,22989,0.6,0.2,0.4,0.1,0.05,0,1180310400,Dutch Windmill


A partir de las preferencias anteriores, podemos inferir algo acerca del usuario PhillyBeer2112. Veamos que recomendaciones se pueden generar para este usuario en particular.

Utilizaremos `recommend()` que utiliza el indice del usuario y lo transpone con la matriz user-item.

In [25]:
X_t = X.T.tocsr()
user_idx = user_mapper[user_id]
recommendations = model.recommend(user_idx, X_t[user_idx])
recommendations

(array([67584, 57402, 47220, 81542, 69197, 48337, 67751, 53229, 90070,
        47309], dtype=int32),
 array([1.3653944, 1.3114238, 1.2922056, 1.2632114, 1.2563404, 1.2099961,
        1.1956567, 1.185583 , 1.1770552, 1.1764188], dtype=float32))

No podemos interpretar los resultados obtenidos pues estan listados los indices. Hagamos una conversion del indice al nombre de las cervezas recomendadas.

In [26]:
for t, r in zip(recommendations[0], recommendations[1]):
    try:
        recommended_title = get_beer_title(t)
        print(recommended_title)
    except:
        continue

Left Hand Oktoberfest
Fort Collins Z Lager
Sweetwater Blue
Sterkens White Ale
Paulaner Original Mnchner Mrzen
Left Hand Deep Cover Brown Ale
Tetleys English Ale
Strohs
Sweetwater Festive Ale


In [34]:
# Encontrar filas que contienen 'Bells Porter' usando str.contains()
result = beers[beers['beer/name'].str.contains('Fort Collins Z Lager', na=False)]

# Mostrar los resultados
print(result)


        beer/beerId             beer/name beer/style  beer/brewerId beer/ABV
2588105       30639  Fort Collins Z Lager     Smoked           3891      4.5


In [35]:
result = beers[beers['beer/name'].str.contains('Tetleys English Ale', na=False)]
print(result)

        beer/beerId            beer/name beer/style  beer/brewerId beer/ABV
2021813        2541  Tetleys English Ale     Bitter            407        5


#### Que podemos decir acerca de las recomendaciones para este usuario?
Al usuario le gusta consumir cervezas con un grado medio de alcohol, además que le gusta probar de distintos estilos por el cuerpo de la cerveza.


## Intentemos con otro usuario y analicemos los resultados obtenidos.

In [36]:
user_id = 'hopdog'

In [37]:
user_ratings = ratings[ratings['review/profileName']==user_id].merge(beers[['beer/beerId', 'beer/name']])
user_ratings = user_ratings.sort_values('review/overall_value_int', ascending=False)
print(f"El numero de cervezas rankeadas por el usuario {user_id} es de: {user_ratings['beer/beerId'].nunique()}")

El numero de cervezas rankeadas por el usuario hopdog es de: 6426


En este caso vemos que el usuario hopdog probó 6426 cervezas y el rating de su favoritas son:

In [38]:
user_ratings = ratings[ratings['review/profileName']==user_id].merge(beers[['beer/beerId', 'beer/name']])
user_ratings = user_ratings.sort_values('review/overall_value_int', ascending=False)
top_5 = user_ratings.head()
top_5

,review/profileName,beer/beerId,review/appearance_value,review/aroma_value,review/palate_value,review/taste_value,review/overall_value,review/overall_value_int,review/time,beer/name
3001,hopdog,46868,0.8,1.0,1.0,1.0,1.00,10,1214524800,3 Fonteinen J & J Oude Geuze Blauw
2997,hopdog,21457,0.8,1.0,1.0,1.0,1.00,10,1183507200,3 Fonteinen Millennium Geuze
553,hopdog,81331,0.8,0.9,0.8,0.8,0.90,9,1228867200,Stone Brandy Barrel Double Bastard
3003,hopdog,37220,0.8,1.0,0.8,0.9,0.95,9,1214524800,3 Fonteinen J & J Oude Geuze Roze
4663,hopdog,48723,0.8,0.9,0.8,0.9,0.90,9,1137628800,Midnight Sun Monks Mistress (Cabernet)


Las cervezas con el menor rating son:

In [39]:
bottom_5 = user_ratings[user_ratings['review/overall_value_int']<5].tail()
bottom_5

,review/profileName,beer/beerId,review/appearance_value,review/aroma_value,review/palate_value,review/taste_value,review/overall_value,review/overall_value_int,review/time,beer/name
1428,hopdog,89777,0.6,0.2,0.4,0.4,0.15,1,1251417600,William Penn Colonial Style Lager
2138,hopdog,79623,0.6,0.2,0.6,0.2,0.10,1,1251417600,Laughing Dog Cold Nose Winter Ale
113,hopdog,33356,0.2,0.3,0.4,0.3,0.15,1,1227052800,Gallo
4397,hopdog,742,0.4,0.1,0.4,0.1,0.05,0,1089849600,Corona Extra
4149,hopdog,75444,0.2,0.1,0.2,0.1,0.05,0,1202428800,Bud Light Chelada


A partir de las preferencias anteriores, podemos inferir algo acerca del usuario hopdog. Veamos que recomendaciones se pueden generar para este usuario en particular.

Utilizaremos `recommend()` que utiliza el indice del usuario y lo transpone con la matriz user-item.

In [40]:
X_t2 = X.T.tocsr()
user_idx = user_mapper[user_id]
recommendations = model.recommend(user_idx, X_t2[user_idx])
recommendations

(array([107987,  59973,  94653,  46386,  67669,  90797,  55638,  60049,
         86533,  58966], dtype=int32),
 array([1.3505028, 1.2971691, 1.2819601, 1.2735821, 1.270067 , 1.2688063,
        1.2611705, 1.2610593, 1.2468184, 1.2408085], dtype=float32))

No podemos interpretar los resultados obtenidos pues estan listados los indices. Hagamos una conversion del indice al nombre de las cervezas recomendadas.

In [41]:
for t, r in zip(recommendations[0], recommendations[1]):
    try:
        recommended_title = get_beer_title(t)
        print(recommended_title)
    except:
        continue

Bert Grants Fresh Hop Ale
Iron Hill Hopilicious IPA
Beer Valley Highway To Ale Barley Wine
Great Divide Hibernation Ale
Left Hand Snow Bound Winter Ale
Stone 07.07.07 Vertical Epic Ale
Unibroue phmre Cassis
Ramstein Blonde
Left Hand Sawtooth Ale
Flying Dog Heller Hound Bock


In [42]:
result = beers[beers['beer/name'].str.contains('Stone 07.07.07 Vertical Epic Ale', na=False)]
print(result)

       beer/beerId                                          beer/name  \
234896       83323  Stone 07.07.07 Vertical Epic Ale (Red Wine Bar...   
250774       74533                   Stone 07.07.07 Vertical Epic Ale   

                beer/style  beer/brewerId beer/ABV  
234896  Belgian Strong Ale             76      8.4  
250774  Belgian Strong Ale             76      8.4  


In [43]:
result = beers[beers['beer/name'].str.contains('Flying Dog Heller Hound Bock', na=False)]
print(result)

        beer/beerId                     beer/name   beer/style  beer/brewerId  \
1101458       32613  Flying Dog Heller Hound Bock  Heller Bock            109   

        beer/ABV  
1101458      6.2  


#### Que podemos decir acerca de las recomendaciones para este usuario?
Al usuario hopdog le gusta consumir cervezas con un grado más alto de alcohol que PhillyBeer2112, además que tiene gusto por las cervezas de la familia 3 Fonteinen J & J Oude y de un estilo más Belgian Strong Ale.
